<a href="https://colab.research.google.com/github/h5ng/GNN/blob/master/torch_geometric_example.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [25]:
!pip install torch-scatter torch-sparse torch-cluster torch-spline-conv -f https://pytorch-geometric.com/whl/torch-1.8.0+cu101.html torch-geometric

Looking in links: https://pytorch-geometric.com/whl/torch-1.8.0+cu101.html


In [26]:
import torch
from torch_geometric.data import Data

In [31]:
edge_index = torch.tensor([[0, 1],
                           [1, 0],
                           [1, 2],
                           [2, 1]], dtype=torch.long)
x = torch.tensor([[-1], [0], [1]], dtype=torch.float)

data = Data(x=x, edge_index=edge_index.t().contiguous())
print(data)
print(data.keys)
print(data.num_nodes)
print(data.num_edges)
print(data.contains_isolated_nodes())
print(data.contains_self_loops())
print(data.is_directed())
print(edge_index.t().contiguous())

Data(edge_index=[2, 4], x=[3, 1])
['x', 'edge_index']
3
4
False
False
False
tensor([[0, 1, 1, 2],
        [1, 0, 2, 1]])


In [28]:
from torch_geometric.datasets import TUDataset

dataset = TUDataset(root='./ENZYMES', name='ENZYMES')
print(dataset)

print(len(dataset))

print(dataset.num_classes) # node 클래스 수

print(dataset.num_node_features) # node 특징 수

print(dataset[0])
# Data(edge_index=[2, 168], x=[37, 3], y=[1])
# 첫번째 그래프에는 3개의 feature를 가진 37개의 노드가 존재 => x=[37, 3]
# 엣지는 총 168 / 2 = 84개의 undirected edge가 있음.
# data object는 정확히 graph-level

ENZYMES(600)
600
6
3
Data(edge_index=[2, 168], x=[37, 3], y=[1])


In [35]:
dataset = dataset.shuffle()
# perm = torch.randperm(len(dataset))
# dataset = dataset[perm]


train_dataset = dataset[:540]
print(train_dataset)
test_dataset = dataset[540:]
print(test_dataset)

ENZYMES(540)
ENZYMES(60)


In [53]:
from torch_geometric.datasets import Planetoid

dataset = Planetoid(root='./Cora', name='CORA')
print(dataset)

print(len(dataset))

print(dataset.num_classes)

print(dataset.num_node_features)

CORA()
1
7
1433


In [54]:
from torch_geometric.datasets import TUDataset
from torch_geometric.data import DataLoader

dataset = TUDataset(root='./ENZYMES', name='ENZYMES')
loader = DataLoader(dataset, batch_size=32, shuffle=True)

for batch in loader:
  print(batch)
  print(batch.num_graphs)

Batch(batch=[1170], edge_index=[2, 4222], x=[1170, 3], y=[32])
32
Batch(batch=[964], edge_index=[2, 3764], x=[964, 3], y=[32])
32
Batch(batch=[955], edge_index=[2, 3836], x=[955, 3], y=[32])
32
Batch(batch=[1073], edge_index=[2, 4252], x=[1073, 3], y=[32])
32
Batch(batch=[1111], edge_index=[2, 4252], x=[1111, 3], y=[32])
32
Batch(batch=[1158], edge_index=[2, 4224], x=[1158, 3], y=[32])
32
Batch(batch=[1044], edge_index=[2, 4136], x=[1044, 3], y=[32])
32
Batch(batch=[957], edge_index=[2, 3632], x=[957, 3], y=[32])
32
Batch(batch=[911], edge_index=[2, 3574], x=[911, 3], y=[32])
32
Batch(batch=[889], edge_index=[2, 3282], x=[889, 3], y=[32])
32
Batch(batch=[1268], edge_index=[2, 4668], x=[1268, 3], y=[32])
32
Batch(batch=[1032], edge_index=[2, 3836], x=[1032, 3], y=[32])
32
Batch(batch=[981], edge_index=[2, 3860], x=[981, 3], y=[32])
32
Batch(batch=[1078], edge_index=[2, 3876], x=[1078, 3], y=[32])
32
Batch(batch=[1036], edge_index=[2, 4060], x=[1036, 3], y=[32])
32
Batch(batch=[1119], ed

In [57]:
from torch_geometric.datasets import Planetoid

dataset = Planetoid(root='./Cora', name='Cora')
dataset

Cora()

In [58]:
  import torch
  import torch.nn.functional as F
  from torch_geometric.nn import GCNConv

  class Net(torch.nn.Module):
    def __init__(self):
      super(Net, self).__init__()
      self.conv1 = GCNConv(dataset.num_node_features, 16)
      self.conv2 = GCNConv(16, dataset.num_classes)

    def forward(self, data):
      x, edge_index = data.x, data.edge_index

      x = self.conv1(x, edge_index)
      x = F.relu(x)
      x = F.dropout(x, training=self.training)
      x = self.conv2(x, edge_index)

      return F.log_softmax(x, dim=1)
      

In [60]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = Net().to(device)
data = dataset[0].to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.01, weight_decay=5e-4)

model.train()
for epoch in range(200):
  optimizer.zero_grad()
  out = model(data)
  loss = F.nll_loss(out[data.train_mask], data.y[data.train_mask])
  loss.backward()
  optimizer.step() 

In [63]:
model.eval()
_, pred = model(data).max(dim=1)
correct = float(pred[data.test_mask].eq(data.y[data.test_mask]).sum().item())
acc = correct / data.test_mask.sum().item()
print('Accuracy: {:.4f}'.format(acc))

Accuracy: 0.7980
